In [ ]:
#nbi:hide_in
#nbi:hide_out
!pip3 install matplotlib
!pip3 install keras
!pip3 install spectral
!pip3 install seaborn
!pip3 install scikit-image
!pip3 install tensorflow
!pip3 install sklearn
!pip3 install tk
!pip3 install -I scikit-learn==0.22.2.post1.

In [1]:
#nbi:hide_in
%matplotlib inline
from bouton import *

# Chargement de l'environnement

In [2]:
#nbi:hide_in
def on_button_clicked1(b):
    with output1:
        output1.clear_output()
        
        chargement_environnement()
        

In [3]:
#nbi:hide_in
button1 = widgets.Button(description="Rechargez l'environnement", button_style='success') # 'success', 'info', 'warning', 'danger' or '')
output1 = widgets.Output()
display(button1, output1)

button1.on_click(on_button_clicked1)

Button(button_style='success', description="Rechargez l'environnement", style=ButtonStyle())

Output()

# Chargement du fichier CSV

*Importer le fichier .csv AVG obtenu grâce au clustering* :

In [4]:
#nbi:hide_in
wl_bis = Upload_CSV()

display(wl_bis)

FileUpload(value={}, button_style='info', description='Fichier CSV')

# VAE

In [5]:
#nbi:hide_in 
bouton_VAE(wl_bis)

Button(button_style='warning', description='VAE', style=ButtonStyle())

Output()